# Cleaning. I followed Chris's cleanning part, so we can have the same dataset.

In [1]:
# Importing necessary modules
import seaborn as sns
import pyspark.sql.functions as F
import pandas as pd
import matplotlib.pyplot as plt
from operator import add
from functools import reduce
import numpy as np
import re
import os
from pyspark.sql.types import StructField, StructType, StringType, LongType, FloatType
from pyspark.sql.functions import *
import random
from pyspark.ml.feature import StandardScaler, VectorAssembler, Imputer, StringIndexer
from pyspark.ml.functions import vector_to_array
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import RFormula
import time

# Setting up visualization
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
cols_to_keep = [
    "Voters_Gender", # cat
#     "Voters_Age", # num
    "Voters_BirthDate", # ignore
    "Residence_Families_HHCount", # num
    "Residence_HHGender_Description", # cat
    "Mailing_Families_HHCount", # num
    "Mailing_HHGender_Description", # cat

#   !! voter party affiliation
    "Parties_Description", 
    
    # cat
    "CommercialData_PropertyType",
    "AddressDistricts_Change_Changed_CD",
    "AddressDistricts_Change_Changed_SD",
    "AddressDistricts_Change_Changed_HD",
    "AddressDistricts_Change_Changed_County",
    
    "Residence_Addresses_Density", # num
    
    # cat
    "CommercialData_EstimatedHHIncome",
    "CommercialData_ISPSA",
    # num
    "CommercialData_AreaMedianEducationYears",
    "CommercialData_AreaMedianHousingValue",
#    "CommercialData_MosaicZ4Global",
    # cat
     "CommercialData_AreaPcntHHMarriedCoupleNoChild",  
     "CommercialData_AreaPcntHHMarriedCoupleWithChild",
     "CommercialData_AreaPcntHHSpanishSpeaking",
     "CommercialData_AreaPcntHHWithChildren",
     "CommercialData_StateIncomeDecile",
#    "Ethnic_Description",
    "EthnicGroups_EthnicGroup1Desc",
    "CommercialData_DwellingType",
    "CommercialData_PresenceOfChildrenCode",
#    "CommercialData_PresenceOfPremCredCrdInHome", ## too many missing
    "CommercialData_DonatesToCharityInHome",
    "CommercialData_DwellingUnitSize",
    "CommercialData_ComputerOwnerInHome",
    "CommercialData_DonatesEnvironmentCauseInHome",
    "CommercialData_Education",
    
#   Don't include because of lookahead bias  
#     "Voters_VotingPerformanceEvenYearGeneral",
#     "Voters_VotingPerformanceEvenYearPrimary",
#     "Voters_VotingPerformanceEvenYearGeneralAndPrimary",
#     "Voters_VotingPerformanceMinorElection",
    
#   Other control variables that expect to be highly associated with outcome:
#     "ElectionReturns_P08CountyTurnoutAllRegisteredVoters",
#     "ElectionReturns_P08CountyTurnoutDemocrats",
#     "ElectionReturns_P08CountyTurnoutRepublicans",
    "General_2000",
    "General_2004",
    "PresidentialPrimary_2000",
    "PresidentialPrimary_2004",
        
#   Outcome variable (indiana law happens in 2005, approved by SCOTUS before presidential election in 2008)
    "General_2008"
]

In [ ]:
# These are the states that do not have strict voter ID laws:
#  'VM2Uniform--CA--2021-05-02',	VM2Uniform--CA--2021-05-02	CA	x	California
#  'VM2Uniform--IL--2021-03-05',	VM2Uniform--IL--2021-03-05	IL	x	Illinois
#  'VM2Uniform--MA--2021-01-19',	VM2Uniform--MA--2021-01-19	MA	x	Massachusetts
#  'VM2Uniform--MD--2021-02-15',	VM2Uniform--MD--2021-02-15	MD	x	Maryland
#  'VM2Uniform--ME--2021-05-28',	VM2Uniform--ME--2021-05-28	ME	x	Maine
#  'VM2Uniform--MN--2021-02-14',	VM2Uniform--MN--2021-02-14	MN	x	Minnesota
#  'VM2Uniform--NC--2021-05-18',	VM2Uniform--NC--2021-05-18	NC	x	North Carolina
#  'VM2Uniform--NE--2021-01-20',	VM2Uniform--NE--2021-01-20	NE	x	Nebraska
#  'VM2Uniform--NJ--2021-03-11',	VM2Uniform--NJ--2021-03-11	NJ	x	New Jersey
#  'VM2Uniform--NM--2021-02-25',	VM2Uniform--NM--2021-02-25	NM	x	New Mexico
#  'VM2Uniform--NV--2021-06-13',	VM2Uniform--NV--2021-06-13	NV	x	Nevada
#  'VM2Uniform--NY--2021-03-15',	VM2Uniform--NY--2021-03-15	NY	x	New York
#  'VM2Uniform--OR--2021-02-05',	VM2Uniform--OR--2021-02-05	OR	x	Oregon
#  'VM2Uniform--PA--2021-05-20',	VM2Uniform--PA--2021-05-20	PA	x	Pennsylvania
#  'VM2Uniform--VT--2021-05-28',	VM2Uniform--VT--2021-05-28	VT	x	Vermont

# For each of these states, I want to pull enough samples to get a total sample of 1/2 M; can increase later

# grab files
states =  [
# For now, just exclude New York and Califonria, because the parquet files take too long to read
'VM2Uniform--VT--2021-05-28', 
'VM2Uniform--IL--2021-03-05',
'VM2Uniform--MA--2021-01-19',
'VM2Uniform--MD--2021-02-15',
'VM2Uniform--ME--2021-05-28',
'VM2Uniform--MN--2021-02-14',
'VM2Uniform--NC--2021-05-18',
'VM2Uniform--NE--2021-01-20',
'VM2Uniform--NJ--2021-03-11',
'VM2Uniform--NM--2021-02-25',
'VM2Uniform--NV--2021-06-13',
'VM2Uniform--OR--2021-02-05',
'VM2Uniform--PA--2021-05-20',
'VM2Uniform--CA--2021-05-02',
'VM2Uniform--NY--2021-03-15',
]

# bucket file path for all state parquet files
gcs_path = 'gs://pstat135-voter-file/VM2Uniform'

# create list of state abbreviations
pattern = re.compile(r"(?<=--)[A-Z]{2}")
state_abvs = re.findall(pattern, ''.join(states))

# do first iteration
print('VM2Uniform--VT--2021-05-28')

# num_per_state = 500

df_ref = spark.read.parquet("/".join([gcs_path, 'VM2Uniform--VT--2021-05-28']))
df_ref = df_ref.select(cols_to_keep)

numrows = {'VM2Uniform--VT--2021-05-28': df_ref.count()}

print("%d" % (numrows['VM2Uniform--VT--2021-05-28']))
    
# percentage_sample = num_per_state / numrows['VM2Uniform--VT--2021-05-28']
    
# df_ref = df_ref.sample(True, percentage_sample, seed = 19480384)
df_ref = df_ref.withColumn('STATE', F.lit(state_abvs[0]))
 
next_states = states[1:]

# do the rest of the iterations
for i, one_state in enumerate(next_states):

    print("%s: " % (one_state), end="")
    
    # read dataframe for one_state
    tmp_ref = spark.read.parquet("/".join([gcs_path, one_state]))
    tmp_ref = tmp_ref.select(cols_to_keep)
    numrows[one_state] = tmp_ref.count()
    print("%d" % (numrows[one_state]))
    
#     percentage_sample = num_per_state / numrows[one_state]
    
#     tmp_ref = tmp_ref.sample(True, percentage_sample, seed = 19480384)
    tmp_ref = tmp_ref.withColumn('STATE', F.lit(state_abvs[i+1]))
    
    df_ref = df_ref.union(tmp_ref)      

df_ref.printSchema()
df_ref.count()

### FUNCTIONS TO CLEAN DATASET

In [8]:
def clean_numeric_categorical(input_df: DataFrame) -> DataFrame:
    
    # remove special symbols ($, %) from relevant columns
    input_df = input_df.withColumn(
        "CommercialData_AreaMedianHousingValue",
        F.expr("substring(CommercialData_AreaMedianHousingValue, 2, length(CommercialData_AreaMedianHousingValue))"))

    pct = ["CommercialData_AreaPcntHHMarriedCoupleNoChild",  
           "CommercialData_AreaPcntHHMarriedCoupleWithChild",
           "CommercialData_AreaPcntHHSpanishSpeaking",
           "CommercialData_AreaPcntHHWithChildren"]

    for c in pct:
        input_df = input_df.withColumn(
            c,
            F.expr(f"substring({c}, 1, length({c})-1)")
        )
    input_df.select(["CommercialData_AreaMedianHousingValue"]+pct).show()

    numeric_cols = [
        'Residence_Families_HHCount',
        'Mailing_Families_HHCount',
        'Residence_Addresses_Density',
        "CommercialData_AreaMedianEducationYears",
        "CommercialData_AreaMedianHousingValue"
    ] + pct

    trinary_cols = [
        'CommercialData_DonatesToCharityInHome',
        'CommercialData_ComputerOwnerInHome',
        'CommercialData_DonatesEnvironmentCauseInHome'
    ]

    binary_cols = []

    dont_touch_cols = [
        "General_2008", 
        "Voters_BirthDate", 
        "General_2000",
        "General_2004",
        "PresidentialPrimary_2000",
        "PresidentialPrimary_2004"
    ]
    
    other_cols = [c for c in input_df.columns if c not in dont_touch_cols]
    other_cols = [c for c in other_cols if c not in (numeric_cols + trinary_cols + binary_cols)]

    categorical_cols = other_cols + binary_cols + trinary_cols

    for c in numeric_cols:
        input_df = input_df.withColumn(c, F.col(c).cast("float").alias(c))
    input_df = input_df.fillna("U", subset= trinary_cols)
    input_df = input_df.fillna("Missing", subset = other_cols)
#   input_df = input_df.fillna("N", subset = binary_cols)
    
    imputer = Imputer(
        inputCols=numeric_cols, 
        outputCols=["{}_imp".format(c) for c in numeric_cols]
    )

    input_df = imputer.fit(input_df).transform(input_df)

    # Impute categorical columns -- maybe it's better to drop these records
    # input_df = input_df.fillna("missing", subset = categorical_cols)

    indexed_cols = [f"{c}_ind" for c in categorical_cols]

    # Ecode categorical variables
    indexer = StringIndexer(inputCols = categorical_cols, outputCols = indexed_cols)
    input_df = indexer.fit(input_df).transform(input_df)
    
    yrs_add = 18
    months_add = 18*12

    # date of national 
    target_month_day_presidential = "11-03"

    # date of presidential primary (ideally we should do this state by state, but this is the date for Indiana's)
    target_month_day_primary = "05-03" 

    input_df = input_df.withColumn("DATE_18", add_months(to_date(col("Voters_BirthDate"),"MM/dd/yyyy"), months_add))
    input_df.select(["Voters_BirthDate", "DATE_18"]).show(10)
    input_df = input_df.dropna(subset = "Voters_BirthDate")
    input_df = input_df.withColumn("YEAR_18", year("DATE_18"))
    input_df = input_df.withColumn("comparator_date_presidential", to_date(concat(col("YEAR_18"), lit("-"), lit(target_month_day_presidential))))
    input_df = input_df.withColumn("comparator_date_primary", to_date(concat(col("YEAR_18"), lit("-"), lit(target_month_day_primary))))

    for election in ["PRESIDENTIAL", "PRIMARY"]:
        input_df = input_df.withColumn(f"YEAR_ELIGIBLE_TO_VOTE_{election}", \
                                    when(col("DATE_18")<=col(f"comparator_date_{election.lower()}"), col("YEAR_18")) \
                                   .otherwise(col("YEAR_18") + 1) \
                                  )

    # check no missing vals:
    input_df.where(col("YEAR_18").isNull()).select("YEAR_18").show(10)

    # get rid of rows where the voter was not old enough to vote in the 2008 general election
    input_df = input_df.filter(col("YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL")<=2008).fillna("N", subset = ["General_2008"])

    # for the 2000 and 2004 general elections, replace with "N" IF the person was old enough to vote at the time

    for election in ["2000", "2004"]:
        input_df = input_df.withColumn(f"General_{election}", \
                                   when((col("YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL")<= int(election)) & \
                                        (col(f"General_{election}").isNull()), "N") \
                                   .otherwise(col(f"General_{election}")) \
                                  )

        input_df = input_df.withColumn(f"PresidentialPrimary_{election}", \
                                   when((col("YEAR_ELIGIBLE_TO_VOTE_PRIMARY")<= int(election)) & \
                                        (col(f"PresidentialPrimary_{election}").isNull()), "N") \
                                   .otherwise(col(f"PresidentialPrimary_{election}")) \
                                  )

    # make the general voting for 2008 a numeric variable; since we've deleted
    # everyone who was not eligible to vote, this can be directly calculated with a 1-0.
    input_df = input_df.withColumn("Voted_General_2008", when(input_df.General_2008 == "Y",1).otherwise(0))
    input_df = input_df.drop("General_2008")
    
    return input_df

In [9]:
df_ref = clean_numeric_categorical(df_ref)
df_ref.printSchema()

+-------------------------------------+---------------------------------------------+-----------------------------------------------+----------------------------------------+-------------------------------------+
|CommercialData_AreaMedianHousingValue|CommercialData_AreaPcntHHMarriedCoupleNoChild|CommercialData_AreaPcntHHMarriedCoupleWithChild|CommercialData_AreaPcntHHSpanishSpeaking|CommercialData_AreaPcntHHWithChildren|
+-------------------------------------+---------------------------------------------+-----------------------------------------------+----------------------------------------+-------------------------------------+
|                              23238.0|                                          39.|                                            20.|                                      1.|                                  29.|
|                              23238.0|                                          39.|                                            20.|               

23/03/19 06:05:55 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------+----------+
|Voters_BirthDate|   DATE_18|
+----------------+----------+
|      12/06/1957|1975-12-06|
|      12/13/1953|1971-12-13|
|      03/24/1977|1995-03-24|
|      08/30/1976|1994-08-30|
|      01/01/1986|2004-01-01|
|      01/01/1963|1981-01-01|
|      01/01/1998|2016-01-01|
|      01/01/1997|2015-01-01|
|      12/28/1963|1981-12-28|
|      11/03/1936|1954-11-03|
+----------------+----------+
only showing top 10 rows



+-------+
|YEAR_18|
+-------+
+-------+

root
 |-- Voters_Gender: string (nullable = false)
 |-- Voters_BirthDate: string (nullable = true)
 |-- Residence_Families_HHCount: float (nullable = true)
 |-- Residence_HHGender_Description: string (nullable = false)
 |-- Mailing_Families_HHCount: float (nullable = true)
 |-- Mailing_HHGender_Description: string (nullable = false)
 |-- Parties_Description: string (nullable = false)
 |-- CommercialData_PropertyType: string (nullable = false)
 |-- AddressDistricts_Change_Changed_CD: string (nullable = false)
 |-- AddressDistricts_Change_Changed_SD: string (nullable = false)
 |-- AddressDistricts_Change_Changed_HD: string (nullable = false)
 |-- AddressDistricts_Change_Changed_County: string (nullable = false)
 |-- Residence_Addresses_Density: float (nullable = true)
 |-- CommercialData_EstimatedHHIncome: string (nullable = false)
 |-- CommercialData_ISPSA: string (nullable = false)
 |-- CommercialData_AreaMedianEducationYears: float (nullable = 

In [10]:
# do the same process for Indiana:
indi = spark.read.parquet("gs://voter-project-235-25/VM2Uniform--IN--2021-01-15_parq")
indi = indi.sample(True, 0.1, seed = 19480384)
indi = indi.select(cols_to_keep)
indi = indi.withColumn("STATE", lit("IN"))
indi = clean_numeric_categorical(indi)
indi.printSchema()

+-------------------------------------+---------------------------------------------+-----------------------------------------------+----------------------------------------+-------------------------------------+
|CommercialData_AreaMedianHousingValue|CommercialData_AreaPcntHHMarriedCoupleNoChild|CommercialData_AreaPcntHHMarriedCoupleWithChild|CommercialData_AreaPcntHHSpanishSpeaking|CommercialData_AreaPcntHHWithChildren|
+-------------------------------------+---------------------------------------------+-----------------------------------------------+----------------------------------------+-------------------------------------+
|                               139950|                                           40|                                             25|                                       1|                                   31|
|                               139950|                                           40|                                             25|               

+----------------+----------+
|Voters_BirthDate|   DATE_18|
+----------------+----------+
|      05/03/1996|2014-05-03|
|      05/03/1996|2014-05-03|
|      12/04/1959|1977-12-04|
|      11/11/1964|1982-11-11|
|      05/21/1925|1943-05-21|
|      07/22/1998|2016-07-22|
|      03/31/1966|1984-03-31|
|      12/10/1983|2001-12-10|
|      01/05/1977|1995-01-05|
|      05/18/1993|2011-05-18|
+----------------+----------+
only showing top 10 rows



+-------+
|YEAR_18|
+-------+
+-------+

root
 |-- Voters_Gender: string (nullable = false)
 |-- Voters_BirthDate: string (nullable = true)
 |-- Residence_Families_HHCount: float (nullable = true)
 |-- Residence_HHGender_Description: string (nullable = false)
 |-- Mailing_Families_HHCount: float (nullable = true)
 |-- Mailing_HHGender_Description: string (nullable = false)
 |-- Parties_Description: string (nullable = false)
 |-- CommercialData_PropertyType: string (nullable = false)
 |-- AddressDistricts_Change_Changed_CD: string (nullable = false)
 |-- AddressDistricts_Change_Changed_SD: string (nullable = false)
 |-- AddressDistricts_Change_Changed_HD: string (nullable = false)
 |-- AddressDistricts_Change_Changed_County: string (nullable = false)
 |-- Residence_Addresses_Density: float (nullable = true)
 |-- CommercialData_EstimatedHHIncome: string (nullable = false)
 |-- CommercialData_ISPSA: integer (nullable = true)
 |-- CommercialData_AreaMedianEducationYears: float (nullable = 

In [11]:
indi.columns

['Voters_Gender',
 'Voters_BirthDate',
 'Residence_Families_HHCount',
 'Residence_HHGender_Description',
 'Mailing_Families_HHCount',
 'Mailing_HHGender_Description',
 'Parties_Description',
 'CommercialData_PropertyType',
 'AddressDistricts_Change_Changed_CD',
 'AddressDistricts_Change_Changed_SD',
 'AddressDistricts_Change_Changed_HD',
 'AddressDistricts_Change_Changed_County',
 'Residence_Addresses_Density',
 'CommercialData_EstimatedHHIncome',
 'CommercialData_ISPSA',
 'CommercialData_AreaMedianEducationYears',
 'CommercialData_AreaMedianHousingValue',
 'CommercialData_AreaPcntHHMarriedCoupleNoChild',
 'CommercialData_AreaPcntHHMarriedCoupleWithChild',
 'CommercialData_AreaPcntHHSpanishSpeaking',
 'CommercialData_AreaPcntHHWithChildren',
 'CommercialData_StateIncomeDecile',
 'EthnicGroups_EthnicGroup1Desc',
 'CommercialData_DwellingType',
 'CommercialData_PresenceOfChildrenCode',
 'CommercialData_DonatesToCharityInHome',
 'CommercialData_DwellingUnitSize',
 'CommercialData_Computer

# logistic regression and propensity score 


In [18]:
df_ref = df_ref.withColumn("LAW", lit(0))

In [20]:
indi = indi.withColumn("LAW", lit(1))
cols_excluded_from_regression = [
        'Voters_BirthDate', # removed this, but KEPT the YEAR that the voter turned 18.
        'STATE',
        'STATE_ind',
        'DATE_18',
        'comparator_date_presidential',
        'comparator_date_primary',
        'YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL',
        'YEAR_ELIGIBLE_TO_VOTE_PRIMARY',
        'Voted_General_2008'
    ]
data = df_ref.union(indi)
data = data.drop(*cols_excluded_from_regression)

In [21]:
## Split data into treatment and control groups

treated = data.filter(data['LAW'] == 1)
control = data.filter(data['LAW'] == 0)

In [22]:
## Create a logistic regression model to predict treatment
supervised = RFormula(formula="LAW ~ .")

In [ ]:
fittedRF = supervised.fit(data)

In [ ]:
prepared = fittedRF.transform(data) 

In [ ]:
featureCols = pd.DataFrame(preparedDF.schema["features"].metadata["ml_attr"]["attrs"]["binary"]+
  preparedDF.schema["features"].metadata["ml_attr"]["attrs"]["numeric"]).sort_values("idx")

featureCols = featureCols.set_index('idx')
featureCols.head()

In [ ]:
lr = LogisticRegression(featuresCol='features', labelCol='LAW')

In [ ]:
lr_model = lr.fit(data)

In [ ]:
# Make predictions on the treatment and control groups
treated_propensity = lr_model.transform(treated)
control_propensity = lr_model.transform(control)

In [ ]:
# Extract the propensity scores
treated_scores = treated_propensity.select(['features', 'probability'])
control_scores = control_propensity.select(['features', 'probability'])

In [ ]:
# Join the propensity scores with the original data
treated_scores = treated_scores.withColumnRenamed('probability', 'treated_propensity')
control_scores = control_scores.withColumnRenamed('probability', 'control_propensity')
data_with_scores = treated_scores.join(control_scores, on='features', how='outer')


In [ ]:
# Calculate the average propensity score in the treatment group
treated_propensity_avg = treated_propensity.agg({'probability': 'avg'}).collect()[0][0]

# Calculate the average propensity score in the control group
control_propensity_avg = control_propensity.agg({'probability': 'avg'}).collect()[0][0]

# Calculate the average treatment effect (ATE)
ATE = treated.mean()['outcome'] - control.mean()['outcome'] / (treated_propensity_avg - control_propensity_avg)